In [0]:
import re
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import OrderedDict
from itertools import chain

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train = pd.read_csv('drive/My Drive/data_PA3/train.csv')
valid= pd.read_csv('drive/My Drive/data_PA3/valid.csv')
test = pd.read_csv('drive/My Drive/data_PA3/partial_test_400.csv')

In [4]:
#finding volcaublary size for English from the training data
train_eng = train["ENG"]
# print(train_temp)
train_eng = np.array(train_eng)
print(train_eng.shape)

p = list(OrderedDict.fromkeys(chain.from_iterable(train_eng)))
input_vocab_size = len(p)+2
print(input_vocab_size)
print(p)

(13122,)
46
['R', ' ', 'A', 'S', 'V', 'I', 'H', 'E', 'D', 'O', 'G', 'N', '_', 'T', 'U', 'M', 'J', 'B', 'L', 'C', 'Y', 'K', 'Z', 'F', '-', 'W', 'P', 'Q', "'", 'X', '^', '/', '6', '(', ')', ',', 'É', '2', '.', '?', '4', 'Á', 'È', '1']


In [5]:
#finding volcaublary size for Hindi from the training data
train_hin = train["HIN"]
# print(train_temp)
train_hin = np.array(train_hin)
print(train_hin.shape)

p = list(OrderedDict.fromkeys(chain.from_iterable(train_hin)))
output_vocab_size = len(p)+2
print(output_vocab_size)
print(p)

(13122,)
86
['र', ' ', 'ा', 'स', 'व', 'ि', 'ह', 'ी', 'द', 'े', 'ग', 'न', '_', 'ो', 'ड', 'श', 'त', '्', 'ु', 'म', 'ज', 'ब', 'ै', 'ल', 'क', 'ॉ', '़', 'य', 'फ', 'ट', 'इ', 'ज़', 'ऑ', 'ं', 'थ', '-', 'ू', 'अ', 'ध', 'प', '\u200d', 'छ', 'च', 'औ', 'ई', 'ॅ', 'आ', 'ख', 'ढ', 'ढ़', 'उ', 'झ', 'ँ', 'भ', 'ौ', 'ष', 'ण', 'घ', 'ठ', 'ए', 'ृ', 'क़', 'ऋ', 'ओ', 'ऐ', '/', 'ड़', '6', '(', ')', ',', 'ञ', "'", '2', '.', 'ऊ', '?', 'फ़', ':', '4', 'ख़', 'ग़', '1', '॥']


In [0]:
inembsize = 256
# embedding_matrix = tf.Variable(tf.random.normal([vocab_size,inembsize],-1.0,1.0)) ##embedding matrix

In [0]:
# embedding_matrix is a tensor of shape [vocabulary_size, embedding size]
# word_embeddings = tf.nn.embedding_lookup(embed_matrix, [3])
# print(word_embeddings)

building encoder


In [0]:
# Build RNN cell
# encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)


In [8]:
batch_size = 8
num_batches = len(train)/batch_size
num_epochs = 3

X_MAX_LENGTH = max([len(train_eng[_]) for _ in range(len(train_eng))])
Y_MAX_LENGTH = max([len(train_hin[_]) for _ in range(len(train_hin))])

num_epochs = 10
state_size = 512  #64, 128, 256 was a good number for linux OS!
encoder_layers = 1
decoder_layers = 1
learning_rate = 0.001
learning_rate_decay = 0.1


state_size = 512

encoder_x = tf.placeholder(dtype=tf.int32, shape=[batch_size, X_MAX_LENGTH]) #[batch_size, X_MAX_LENGTH]
decoder_x = tf.placeholder(dtype=tf.int32, shape=[batch_size, Y_MAX_LENGTH, output_vocab_size]) #[batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE]
y = tf.placeholder(dtype=tf.float32, shape=[batch_size, Y_MAX_LENGTH, output_vocab_size])#[batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE]
init_state = tf.placeholder(tf.float32, [encoder_layers, 2, batch_size, state_size])

print('[CONFIG] LSTM Sequence-to-Sequence Translation Model Configuration:')
print('[CONFIG] Batch size: {}'.format(batch_size))
print('[CONFIG] Number of Hidden Layers (Encoder/Decoder): {}/{}'.format(encoder_layers, decoder_layers))
print('[CONFIG] Hidden Layer State Size: {}'.format(state_size))
print('[CONFIG] X Placeholder Shape: [{}, {}]'.format('None', 'None'))
print('[CONFIG] Y Placeholder Shape: [{}, {}, {}]\n'.format('None', 1, output_vocab_size))
print(X_MAX_LENGTH)
print(Y_MAX_LENGTH)

[CONFIG] LSTM Sequence-to-Sequence Translation Model Configuration:
[CONFIG] Batch size: 8
[CONFIG] Number of Hidden Layers (Encoder/Decoder): 1/1
[CONFIG] Hidden Layer State Size: 512
[CONFIG] X Placeholder Shape: [None, None]
[CONFIG] Y Placeholder Shape: [None, 1, 86]

121
123


In [0]:
########
#
# METHOD: vectorize_data()
# DESCRIPTION: Transform sequences from word_id form into one_hot form.
# PARAMS:
#           word_sentences: sequences of word_ids
#           max_length: MAX_LENGTH of sentences w/ padding.
#           word_to_idx: data structure containing word mappings to their id representation.
# RETURNS:
#           sequences: Batch of one_hot sequences.
#
########
def vectorize_data(word_sentences, max_length, word_to_idx):
    sequences = np.zeros((len(word_sentences), max_length, len(word_to_idx)), dtype=float)
    for i, sentence in enumerate(word_sentences):
        for j, word in enumerate(sentence):
            sequences[i, j, word] = 1.
    return sequences


In [0]:
########
#
# METHOD: generate_text()
# DESCRIPTION: Translate LSTM output probabilities into target vocabulary using predefined dictionary.
#              *Avoid printing 'UNK' characters!*
# PARAMS:
#           prediction: array of probabilistic values that correspond to LSTM estimation outputs.
#           batch_size: number of parallel sequences processed.
#           length: MAX_LENGTH of each sequence w/ padding.
#           vocab_size: Size of target vocabulary - corresponds to number of classes within network estimations.
#           idx_to_word: Word ids mapped to corresponding words for translation.
# RETURNS:
#           batch_sequence: Batch of translated sentences.
#
########
def generate_text(prediction, batch_size, length, vocab_size, idx_to_word):

    batch_softmax = np.reshape(prediction, [batch_size, length, vocab_size])
    batch_sentence = []

    for sequence in batch_softmax:
        word_sequence = ''
        for char in sequence:
            vector_position = np.argmax(char)
            y_word = idx_to_word[vector_position]
            if y_word != 'ZERO':
                word_sequence = word_sequence + y_word + ' '
            else:
                word_sequence = word_sequence + ''
        batch_sentence.append(word_sequence)

    return batch_sentence


In [0]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [0]:
def lstm_model(data):

    with tf.device('/cpu:0'):

        with tf.variable_scope('encoder_word_embeddings'):

            word_embeddings = tf.get_variable('encoder_word_embeddings', [input_vocab_size, inembsize])
            encoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, encoder_x)
            encoder_embedded_word_ids = tf.reshape(encoder_embedded_word_ids, [-1, X_MAX_LENGTH, inembsize])

        with tf.variable_scope('encoder'):
            ####
            #
            # LSTM ENCODER
            #
            ####
            # Forward passes
            state_per_layer_list = tf.unstack(init_state, axis=0)
            rnn_tuple_state = tuple(
                [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])
                for idx in range(encoder_layers)])

            encoder_stacked_cell = []

            for _ in range(encoder_layers):
                encoder_single_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
                if _ == 0:
                    with tf.name_scope('encoder_dropout') as scope:
                        encoder_single_cell = tf.nn.rnn_cell.DropoutWrapper(encoder_single_cell,
                                                        output_keep_prob=0.75)  # add dropout to first LSTM layer only.
                encoder_stacked_cell.append(encoder_single_cell)

            encoder_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_stacked_cell, state_is_tuple=True)

            encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(cell=encoder_cell,
                                                             inputs=encoder_embedded_word_ids,
                                                             initial_state=rnn_tuple_state)

            del encoder_outputs, encoder_cell, state_per_layer_list, encoder_stacked_cell
            
            print("HIHIHIHI")
            print(encoder_final_state)
            
#             print("HIHIHIHI")
#             print(encoder_final_state.shape)
            

        with tf.variable_scope('decoder_word_embeddings'):

            word_embeddings = tf.get_variable('decoder_word_embeddings', [output_vocab_size, inembsize])
            decoder_embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, decoder_x)
            decoder_embedded_word_ids = tf.reshape(decoder_embedded_word_ids,
                                                        [-1, Y_MAX_LENGTH, output_vocab_size * inembsize])

        with tf.variable_scope('decoder'):
            ####
            #
            # LSTM DECODER
            #
            ####
            decoder_stacked_cell = []
            for _ in range(decoder_layers):
                decoder_single_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
                if _ == 0:
                    with tf.name_scope('decoder_dropout') as scope:
                        decoder_single_cell = tf.nn.rnn_cell.DropoutWrapper(decoder_single_cell,
                                                    output_keep_prob=0.75)  # add dropout to first LSTM layer only.
                decoder_stacked_cell.append(decoder_single_cell)

            decoder_cell = tf.nn.rnn_cell.MultiRNNCell(decoder_stacked_cell, state_is_tuple=True)

            decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(cell=decoder_cell,
                                                             inputs=decoder_embedded_word_ids,
                                                             initial_state=encoder_final_state)

            outputs = tf.reshape(decoder_outputs, [-1, state_size])

            with tf.name_scope('decoder_hidden_states') as scope:
                W2 = tf.Variable(tf.random_normal([state_size, output_vocab_size]), dtype=tf.float32)
                variable_summaries(W2)
                b2 = tf.Variable(tf.zeros([1, output_vocab_size]), dtype=tf.float32)
                variable_summaries(b2)

                logits = tf.matmul(outputs, W2) + b2 # Broadcasted addition
                tf.summary.histogram('pre_activations', logits)
                prediction = tf.nn.softmax(logits)
                tf.summary.histogram('activations', prediction)
            labels = y

    return decoder_final_state, labels , logits, prediction

In [0]:

def load_data(inputs = train_eng, labels = train_hin, vocabulary_size = 100):

    print('[INFO] Importing RAW Inputs and Labels...')
    x_data_source = open(inputs, 'r')
    y_data_source = open(labels, 'r')

    # Feed input data in backwards for better translation performance.
    x_data = (tf.compat.as_str(x_data_source.read()).lower().split(' '))
    x_data = [x_data[_].split(' ')[::-1] for _ in range(len(x_data))]
    X_DATA_SIZE = len(x_data)
    print('[NOTIFICATION] Here is a snippet of the Inputs!')
    print(x_data[0:2])

    y_data = (tf.compat.as_str(y_data_source.read()).lower().split(' '))
    y_data = [y_data[_].split(' ') for _ in range(len(y_data))]
    Y_DATA_SIZE = len(y_data)
    print('[NOTIFICATION] Here is a snippet of the Labels!')
    print(y_data[0:2])


    print('[INFO] Creating Dictionary Indexes...')
    #Word Dictionary
    x_word_list = []
    y_word_list = []
    [x_word_list.extend(x_data[_]) for _ in range(len(x_data))]
    [y_word_list.extend(y_data[_]) for _ in range(len(y_data))]

    x_word_dictionary = []
    y_word_dictionary = []
    print('[INFO] Finding most common vocabulary...')
    print('[INFO] The common vocabulary threshold is set to: {}'.format(vocabulary_size))
    x_word_dictionary.extend(collections.Counter(x_word_list).most_common(44))
    y_word_dictionary.extend(collections.Counter(y_word_list).most_common(84))

    print('[INFO] Clearing up memory...')
    del x_word_list, y_word_list

    print('[INFO] Mapping vocabulary to idx...')
    #word/idx mapping
    x_idx_to_word = [word[0] for idx, word in enumerate(x_word_dictionary)]
    x_idx_to_word.insert(0, '#')
    x_idx_to_word.append('$')
    print('[NOTIFICATION] Here is a snippet of the Input Dictionary!')
    print(x_idx_to_word[0:20])

    y_idx_to_word = [word[0] for idx, word in enumerate(y_word_dictionary)]
    y_idx_to_word.insert(0, '#') #at the first 
    y_idx_to_word.append('$') #at the last
    print('[NOTIFICATION] Here is a snippet of the Label Dictionary!')
    print(y_idx_to_word[0:20])

    x_word_to_idx = {word:ix for ix, word in enumerate(x_idx_to_word)}
    y_word_to_idx = {word: ix for ix, word in enumerate(y_idx_to_word)}

    X_VOCAB_SIZE = len(x_word_dictionary) + 2
    Y_VOCAB_SIZE = len(y_word_dictionary) + 2

    print('[INFO] Converting vocabulary to index value...')
    # Converting each character to its index value
    for i, word in enumerate(x_data):
        for j, charac in enumerate(word):
            if charac in x_word_to_idx:
                x_data[i][j] = x_word_to_idx[charac]
            else:
                x_data[i][j] = x_word_to_idx['$']
    print('[INFO] Input conversion complete!')
    print('[NOTIFICATION] Here is a snippet of the Input Index Values!')
    print(x_data[0:2])

    for i, word in enumerate(y_data):
        for j, charac in enumerate(word):
            if charac in y_word_to_idx:
                y_data[i][j] = y_word_to_idx[charac]
            else:
                y_data[i][j] = y_word_to_idx['$']
    print('[INFO] Label conversion complete!')
    print('[NOTIFICATION] Here is a snippet of the Label Index Values!')
    print(y_data[0:2])

    X_MAX_LENGTH = max([len(x_data[_]) for _ in range(len(x_data))])
    Y_MAX_LENGTH = max([len(y_data[_]) for _ in range(len(y_data))])

    del x_data, y_data

    return X_VOCAB_SIZE, Y_VOCAB_SIZE, x_idx_to_word, x_word_to_idx, y_idx_to_word, y_word_to_idx, X_MAX_LENGTH, Y_MAX_LENGTH, X_DATA_SIZE


In [0]:
def load_data_by_batch(batch_number, x_word_to_idx, y_word_to_idx, inputs = train_eng, labels = train_hin,
                       batch_size = batch_size):

    start_batch = batch_number * batch_size
    end_batch = start_batch + batch_size

    print('\n[INFO] Importing RAW Inputs and Labels for batch...')
    x_data_source = open(inputs, 'r')
    y_data_source = open(labels, 'r')

    #Feed input data in backwards for better translation performance.
    x_data = (tf.compat.as_str(x_data_source.read()).lower().split(' '))
    x_data = [x_data[_].split(' ')[::-1] for _ in range(len(x_data))]
    x_data = x_data[start_batch:end_batch]

    y_data = (tf.compat.as_str(y_data_source.read()).lower().split(' '))
    y_data = [y_data[_].split(' ') for _ in range(len(y_data))]
    y_data = y_data[start_batch:end_batch]

    # Converting each character to its index value
    for i, word in enumerate(x_data):
        for j, charac in enumerate(word):
            if charac in x_word_to_idx:
                x_data[i][j] = x_word_to_idx[charac]
            else:
                x_data[i][j] = x_word_to_idx['$']

    for i, word in enumerate(y_data):
        for j, charac in enumerate(word):
            if charac in y_word_to_idx:
                y_data[i][j] = y_word_to_idx[charac]
            else:
                y_data[i][j] = y_word_to_idx['$']

    return x_data, y_data

In [0]:
restore = False
ckpt_model_directory = 'drive/My Drive/data_files/training/saved_models'

In [0]:
def lstm_train():

    #Initialize CPU config..
#     config = config_tensorflow_hardware()

    current_state, labels, logits, prediction = lstm_model(encoder_x)

    with tf.name_scope('cross_entropy') as scope:
        cost = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
        total_cost = tf.reduce_mean(cost)
        tf.summary.scalar('cross-entropy', total_cost)

    with tf.name_scope('optimizer') as scope:
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_cost)

    #Keep previous 30 model checkpoints..
    saver = tf.train.Saver(max_to_keep=30)

    with tf.Session() as sess:

        merged = tf.summary.merge_all()
#         writer = tf.summary.FileWriter(logs_path, graph=sess.graph)

        sess.run(tf.global_variables_initializer())

        epoch = 0

#         if restore:
#             print('Loading variables from {}'.format(ckpt_model_directory + nn_config.meta_file + nn_config.meta_num + nn_config.meta_ext))
#             saver.restore(sess, nn_config.ckpt_model_directory + nn_config.meta_file + nn_config.meta_num)

        _current_state = np.zeros((encoder_layers, 2, batch_size, state_size))

        while True: #Unlimited epochs for Training

            epoch_loss = 0

            for batch in range(num_batches):

                batch_x, batch_y = load_data_by_batch(batch, x_word_to_idx, y_word_to_idx)

                batch_loss = 0

                print('[INFO] Zero padding RAW batch data...'.format(batch))
                for _ in range(len(batch_x)):
                    x_length = len(batch_x[_])
                    y_length = len(batch_y[_])

                    batch_x[_].extend(np.zeros([X_MAX_LENGTH - x_length], dtype=int))
                    batch_y[_].extend(np.zeros([Y_MAX_LENGTH - y_length], dtype=int))

                y_one_hot = vectorize_data(batch_y, Y_MAX_LENGTH, y_word_to_idx)

                del batch_y

                print('[INFO] Passing batch values into Encoder/Decoder for LSTM network optimization...'.format(batch))

                summary, _total_cost, _train_step, _current_state, _prediction_series = sess.run(
                [merged, total_cost, optimizer, current_state, prediction],
                feed_dict={
                        encoder_x: batch_x,
                        decoder_x: y_one_hot,
                        y: y_one_hot,
                        init_state: _current_state
                    })

#                 writer.add_summary(summary, epoch)

                if batch % 2 == 0:

                    print('[INFO] Translating Softmax Probabilities...')

                    text = generate_text(_prediction_series, batch_size, Y_MAX_LENGTH, Y_VOCAB_SIZE, y_idx_to_word)

                    if epoch % 5 == 0:
                        save_output_path = 'data_files/training/encoder-decoder-output/TRAINING_TRANSLATION_OUTPUT_EPOCH_{}_STATE-SIZE_{}_NUM-LAYERS_{}_LEARNING-RATE{}_EMBEDDING-SIZE_{}.txt'.format(epoch, state_size, decoder_layers, nn_config.learning_rate, nn_config.embedding_size)

                        with open(save_output_path,'wb') as f:
                         for item in text:
                             f.write("%s\n" % item)
                        print('[NOTIFICATION] Translation file saved to: {}'.format(save_output_path))

                    print('[INFO]: Batch {} optimized output for Epoch {}:'.format(batch, epoch))
                    for _ in range(batch_size):
                        print(text[_])

                batch_loss += _total_cost
                epoch_loss += _total_cost

                print('\n[STATUS] Batch {}/{} complete! Batch Loss: {}'.format(batch, num_batches, batch_loss))

            if epoch % 1 == 0:

                save_model_path = "data_files/training/saved_models/translation_model"
                if epoch == 0:
                    save_path = saver.save(sess, save_model_path, epoch)
                else:
                    save_path = saver.save(sess, save_model_path, epoch, write_meta_graph=False)

                print("[STATUS] Model ckpt saved in file: %s" % save_path)

            print('[STATUS] Epoch {} complete! Epoch Loss: {}'.format(epoch, epoch_loss))

            saver.save(sess, logs_path + '/model.ckpt', epoch)
            epoch+=1

# lstm_train()

In [17]:
lstm_train()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
HIHIHIHI
(LSTMStateTuple(c=<tf.Tensor 'encoder/rnn/while/Exit_3:0' shape=(8, 512) dtype=float32>, h=<tf.Tensor 'encoder/rnn/while/Exit_4:0' shape=(8, 512) dtype=float32>),)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.


TypeError: ignored